# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN
print(stocks)

    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[505 rows x 1 columns]


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
# getting info for one stock
#https://iexcloud.io/docs/api/#key-stats
symbol = "AAPL"
url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(url)
print(data)

<Response [200]>


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data = data.json()

In [5]:
print(data)
print(data['maxChangePercent'])

{'companyName': 'Apple Inc', 'marketcap': 2277820435459, 'week52high': 141.51, 'week52low': 55.75, 'week52change': 0.8221077626748624, 'sharesOutstanding': 17240107687, 'float': 0, 'avg10Volume': 102684790, 'avg30Volume': 110164276, 'day200MovingAvg': 113.4, 'day50MovingAvg': 125.82, 'employees': 140219, 'ttmEPS': 3.29, 'ttmDividendRate': 0.8224977135065342, 'dividendYield': 0.006084636076808246, 'nextDividendDate': '0', 'exDividendDate': '2020-10-28', 'nextEarningsDate': '0', 'peRatio': 39.52936190638737, 'beta': 1.1949094471049002, 'maxChangePercent': 51.34580093921731, 'year5ChangePercent': 4.479053263433737, 'year2ChangePercent': 2.559416715254451, 'year1ChangePercent': 0.8325504139537684, 'ytdChangePercent': 0, 'month6ChangePercent': 0.4689963994762513, 'month3ChangePercent': 0.13889263362475818, 'month1ChangePercent': 0.08171389994749748, 'day30ChangePercent': 0.08174959345900007, 'day5ChangePercent': 0.005705501167527965}
51.34580093921731


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
#symbol_groups = list(chunks(stocks['Ticker'], 100))
#symbol_strings = []
#for i in range(0, len(symbol_groups)):
#    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
def batch(symbol_list):
    url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={','.join(symbol_list)}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}"
    return requests.get(url).json()


In [8]:
test = chunks(stocks['Ticker'], 5)
batch(list(test)[0])

{'A': {'stats': {'companyName': 'Agilent Technologies Inc.',
   'marketcap': 38077453854,
   'week52high': 123.2,
   'week52low': 63.85,
   'week52change': 0.41664180846982446,
   'sharesOutstanding': 318605349,
   'float': 0,
   'avg10Volume': 1073360,
   'avg30Volume': 1687802,
   'day200MovingAvg': 106.81,
   'day50MovingAvg': 118.17,
   'employees': 13735,
   'ttmEPS': 2.32,
   'ttmDividendRate': 0.5464108624112776,
   'dividendYield': 0.004723961482444714,
   'nextDividendDate': '2020-12-20',
   'exDividendDate': '2020-09-26',
   'nextEarningsDate': '0',
   'peRatio': 54.58875197903125,
   'beta': 0.927123184181169,
   'maxChangePercent': 4.275543842959706,
   'year5ChangePercent': 2.052338080769594,
   'year2ChangePercent': 0.807684579368205,
   'year1ChangePercent': 0.4067847221397241,
   'ytdChangePercent': 0,
   'month6ChangePercent': 0.3495666870941473,
   'month3ChangePercent': 0.17702317613550442,
   'month1ChangePercent': 0.027199418894241295,
   'day30ChangePercent': 0.03

In [9]:
df = pd.DataFrame(columns = my_columns)
for symbols in chunks(stocks['Ticker'], 100):
    data = batch(symbols)
    for sym in symbols:
        df = df.append(
            pd.Series(
                [sym, 
                 data[sym]['quote']['latestPrice'], 
                 data[sym]['stats']['year1ChangePercent'],
                 'N/A'
                ],
                index = my_columns
            ),
            ignore_index=True
        )


In [10]:
print(df)

    Ticker   Price One-Year Price Return Number of Shares to Buy
0        A  119.05              0.405649                     N/A
1      AAL   16.43             -0.454680                     N/A
2      AAP  158.51             -0.009208                     N/A
3     AAPL  134.21              0.845586                     N/A
4     ABBV  108.37              0.288123                     N/A
..     ...     ...                   ...                     ...
500    YUM  111.06              0.099590                     N/A
501    ZBH  156.37              0.039316                     N/A
502   ZBRA  386.09              0.526885                     N/A
503   ZION   43.50             -0.134099                     N/A
504    ZTS  166.60              0.259049                     N/A

[505 rows x 4 columns]


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [11]:
top = df.sort_values(by='One-Year Price Return', ascending=False).head(50)
print(top)

    Ticker    Price One-Year Price Return Number of Shares to Buy
78    CARR    39.44              2.209539                     N/A
345   NVDA   533.20              1.238150                     N/A
387   PYPL   242.00              1.184557                     N/A
23     ALB   151.28              1.087878                     N/A
275     LB    37.59              1.086591                     N/A
31     AMD    93.29              1.034061                     N/A
179    FCX    26.84              1.002623                     N/A
85    CDNS   137.81              0.981903                     N/A
24    ALGN   548.28              0.959275                     N/A
339    NOW   571.22              0.956899                     N/A
236   IDXX   524.63              0.929909                     N/A
6     ABMD   331.30              0.919835                     N/A
490    WST   288.47              0.891263                     N/A
415   SNPS   269.17              0.882335                     N/A
3     AAPL

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [12]:
def num_shares(df, portfolio_size):
    stock_val = portfolio_size / len(df)
    for i in df.index:
        df.loc[i, 'Number of Shares to Buy'] = math.floor(stock_val / df.loc[i, 'Price'])

In [13]:
num_shares(top, 1000000)
print(top)

    Ticker    Price One-Year Price Return Number of Shares to Buy
78    CARR    39.44              2.209539                     507
345   NVDA   533.20              1.238150                      37
387   PYPL   242.00              1.184557                      82
23     ALB   151.28              1.087878                     132
275     LB    37.59              1.086591                     532
31     AMD    93.29              1.034061                     214
179    FCX    26.84              1.002623                     745
85    CDNS   137.81              0.981903                     145
24    ALGN   548.28              0.959275                      36
339    NOW   571.22              0.956899                      35
236   IDXX   524.63              0.929909                      38
6     ABMD   331.30              0.919835                      60
490    WST   288.47              0.891263                      69
415   SNPS   269.17              0.882335                      74
3     AAPL

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [35]:
# create columns
return_columns = ['1-month price return', 
                  '3-month price return',
                  '6-month price return',
                  '1-year price return',]

percentile_columns = ['1-month price percentile', 
                     '3-month price percentile', 
                     '6-month price percentile', 
                     '1-year price percentile', ]


my_columns = ['Ticker', 'Price'] + return_columns + percentile_columns + [         
              'HQM Score',
              'Number of Shares to Buy']


df = pd.DataFrame(columns=my_columns)



In [36]:
df.info

<bound method DataFrame.info of Empty DataFrame
Columns: [Ticker, Price, 1-month price return, 3-month price return, 6-month price return, 1-year price return, 1-month price percentile, 3-month price percentile, 6-month price percentile, 1-year price percentile, HQM Score, Number of Shares to Buy]
Index: []>

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [37]:
# get data from API
df = pd.DataFrame(columns = my_columns)
for symbols in chunks(stocks['Ticker'], 100):
    data = batch(symbols)
    for sym in symbols:
        df = df.append(
            pd.Series(
                [sym, 
                 data[sym]['quote']['latestPrice'], 
                 data[sym]['stats']['month1ChangePercent'],  
                 data[sym]['stats']['month3ChangePercent'],   
                 data[sym]['stats']['month6ChangePercent'],  
                 data[sym]['stats']['year1ChangePercent']] + [None] * 6,
                index = my_columns
            ),
            ignore_index=True
        )

In [38]:
print(df.info())

# convert return columns and percentile columns to numeric type
df[return_columns] = df[return_columns].apply(pd.to_numeric)
df[percentile_columns] = df[percentile_columns].apply(pd.to_numeric)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Ticker                    505 non-null    object 
 1   Price                     505 non-null    float64
 2   1-month price return      501 non-null    object 
 3   3-month price return      501 non-null    object 
 4   6-month price return      501 non-null    object 
 5   1-year price return       501 non-null    object 
 6   1-month price percentile  0 non-null      object 
 7   3-month price percentile  0 non-null      object 
 8   6-month price percentile  0 non-null      object 
 9   1-year price percentile   0 non-null      object 
 10  HQM Score                 0 non-null      object 
 11  Number of Shares to Buy   0 non-null      object 
dtypes: float64(1), object(11)
memory usage: 47.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 t

In [39]:
# in order to calculate percentiles, need to check for 'None' values and replace with np.nan
for i in df.index:
    for col in return_columns:
        if df.loc[i, col] is None:
            df.loc[i, col] = np.nan

In [40]:
for return_col, percentile_col in zip(return_columns, percentile_columns):
    df[percentile_col] = df[return_col].rank(pct=True)
    

In [41]:
df

,Ticker,Price,1-month price return,3-month price return,6-month price return,1-year price return,1-month price percentile,3-month price percentile,6-month price percentile,1-year price percentile,HQM Score,Number of Shares to Buy
0,A,122.74,0.028385,0.175238,0.358196,0.403229,0.546906,0.518962,0.684631,0.858283,None,None
1,AAL,16.02,0.105738,0.262484,0.239866,-0.451079,0.894212,0.698603,0.475050,0.025948,None,None
2,AAP,161.78,0.034597,0.019099,0.114070,-0.009285,0.594810,0.177645,0.261477,0.355289,None,None
3,AAPL,136.69,0.082026,0.142341,0.467598,0.827510,0.828343,0.449102,0.820359,0.972056,None,None
4,ABBV,112.38,0.030444,0.246534,0.113148,0.282895,0.560878,0.652695,0.257485,0.762475,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.13,0.021380,0.192508,0.271462,0.101937,0.485030,0.562874,0.528942,0.516966,None,None
501,ZBH,157.69,0.058451,0.125228,0.298068,0.039144,0.718563,0.411178,0.586826,0.429142,None,None
502,ZBRA,390.63,0.033878,0.506732,0.547907,0.521079,0.584830,0.940120,0.898204,0.920160,None,None
503,ZION,44.34,0.086185,0.510204,0.365990,-0.135350,0.844311,0.942116,0.702595,0.189621,None,None


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [42]:
for i in df.index:
    mean = 0
    try:
        for col in percentile_columns:
            mean += df.loc[i, col]
        mean /= len(percentile_columns)
    except TypeError:
        mean = None
    df.loc[i, 'HQM Score'] = mean

In [43]:
df

,Ticker,Price,1-month price return,3-month price return,6-month price return,1-year price return,1-month price percentile,3-month price percentile,6-month price percentile,1-year price percentile,HQM Score,Number of Shares to Buy
0,A,122.74,0.028385,0.175238,0.358196,0.403229,0.546906,0.518962,0.684631,0.858283,0.652196,None
1,AAL,16.02,0.105738,0.262484,0.239866,-0.451079,0.894212,0.698603,0.475050,0.025948,0.523453,None
2,AAP,161.78,0.034597,0.019099,0.114070,-0.009285,0.594810,0.177645,0.261477,0.355289,0.347305,None
3,AAPL,136.69,0.082026,0.142341,0.467598,0.827510,0.828343,0.449102,0.820359,0.972056,0.767465,None
4,ABBV,112.38,0.030444,0.246534,0.113148,0.282895,0.560878,0.652695,0.257485,0.762475,0.558383,None
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.13,0.021380,0.192508,0.271462,0.101937,0.485030,0.562874,0.528942,0.516966,0.523453,None
501,ZBH,157.69,0.058451,0.125228,0.298068,0.039144,0.718563,0.411178,0.586826,0.429142,0.536427,None
502,ZBRA,390.63,0.033878,0.506732,0.547907,0.521079,0.584830,0.940120,0.898204,0.920160,0.835828,None
503,ZION,44.34,0.086185,0.510204,0.365990,-0.135350,0.844311,0.942116,0.702595,0.189621,0.669661,None


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [44]:
top = df.sort_values(by='HQM Score', ascending=False).head(50)

In [45]:
top

,Ticker,Price,1-month price return,3-month price return,6-month price return,1-year price return,1-month price percentile,3-month price percentile,6-month price percentile,1-year price percentile,HQM Score,Number of Shares to Buy
179,FCX,27.14,0.100751,0.690139,1.269774,1.013193,0.880240,0.978044,0.996008,0.988024,0.960579,None
23,ALB,153.18,0.091905,0.727674,0.966306,1.090723,0.862275,0.988024,0.988024,0.992016,0.957585,None
24,ALGN,536.96,0.071970,0.703601,0.976373,0.946873,0.792415,0.980040,0.990020,0.982036,0.936128,None
410,SIVB,404.52,0.092930,0.589834,0.882798,0.558870,0.868263,0.968064,0.978044,0.930140,0.936128,None
324,MU,77.46,0.121721,0.586457,0.519655,0.406576,0.928144,0.964072,0.878244,0.860279,0.907685,None
134,DIS,187.50,0.221441,0.471355,0.609891,0.257312,0.990020,0.928144,0.924152,0.720559,0.890719,None
358,PAYC,460.10,0.088242,0.431849,0.432327,0.735761,0.854291,0.904192,0.784431,0.956088,0.874750,None
441,TPR,31.41,0.097408,0.963922,1.415075,0.176554,0.876248,0.996008,0.998004,0.618762,0.872255,None
147,DVA,120.40,0.081667,0.423501,0.468919,0.577128,0.822355,0.896208,0.826347,0.936128,0.870259,None
28,ALXN,163.91,0.292045,0.351676,0.406476,0.457499,0.998004,0.826347,0.744511,0.892216,0.865269,None


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [46]:
num_shares(top, 10000)

In [28]:
top

,Ticker,Price,1-month price return,3-month price return,6-month price return,1-year price return,1-month price percentile,3-month price percentile,6-month price percentile,1-year price percentile,HQM Score,Number of Shares to Buy
179,FCX,26.69,0.102691,0.704129,1.283762,0.994918,0.882236,0.984032,0.996008,0.986028,0.962076,7
23,ALB,152.01,0.092914,0.714768,0.985995,1.060498,0.866267,0.986028,0.990020,0.992016,0.958583,1
410,SIVB,402.45,0.092441,0.596862,0.880262,0.549633,0.864271,0.970060,0.976048,0.928144,0.934631,0
24,ALGN,536.92,0.069420,0.682636,0.951789,0.924965,0.780439,0.978044,0.988024,0.980040,0.931637,0
324,MU,75.21,0.124349,0.580924,0.530669,0.401187,0.928144,0.964072,0.882236,0.854291,0.907186,2
134,DIS,189.37,0.216246,0.471709,0.629221,0.253815,0.990020,0.930140,0.936128,0.718563,0.893713,1
441,TPR,32.09,0.097348,0.960261,1.432030,0.173419,0.876248,0.996008,0.998004,0.620758,0.872754,6
28,ALXN,162.45,0.286239,0.364410,0.407794,0.445443,0.998004,0.840319,0.748503,0.886228,0.868263,1
358,PAYC,454.49,0.090321,0.415932,0.419683,0.728441,0.856287,0.892216,0.764471,0.956088,0.867265,0
147,DVA,121.30,0.082191,0.419061,0.463540,0.566841,0.826347,0.896208,0.810379,0.934132,0.866766,1


In [48]:
print(top.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 179 to 387
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Ticker                    50 non-null     object 
 1   Price                     50 non-null     float64
 2   1-month price return      50 non-null     float64
 3   3-month price return      50 non-null     float64
 4   6-month price return      50 non-null     float64
 5   1-year price return       50 non-null     float64
 6   1-month price percentile  50 non-null     float64
 7   3-month price percentile  50 non-null     float64
 8   6-month price percentile  50 non-null     float64
 9   1-year price percentile   50 non-null     float64
 10  HQM Score                 50 non-null     object 
 11  Number of Shares to Buy   50 non-null     object 
dtypes: float64(9), object(3)
memory usage: 7.1+ KB
None


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [69]:
writer = pd.ExcelWriter('HQM_trades.xlsx', engine='xlsxwriter')
top.to_excel(writer, 'Recommended trades', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [70]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [71]:
my_columns

['Ticker',
 'Price',
 '1-month price return',
 '3-month price return',
 '6-month price return',
 '1-year price return',
 '1-month price percentile',
 '3-month price percentile',
 '6-month price percentile',
 '1-year price percentile',
 'HQM Score',
 'Number of Shares to Buy']

In [72]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Stock Price', dollar_template],
    'C': ['1-month price return', percent_template],
    'D': ['3-month price return', percent_template], 
    'E': ['6-month price return', percent_template],
    'F': ['1-year price return', percent_template],
    'G': ['1-month price percentile', percent_template],
    'H': ['3-month price percentile', percent_template], 
    'I': ['6-month price percentile', percent_template],
    'J': ['1-year price percentile', percent_template],
    'K': ['HQM score', percent_template],    
    'L': ['Number of Shares to Buy', integer_template]
}

In [73]:
for column in column_formats:
    writer.sheets['Recommended trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended trades'].write(f"{column}1", column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [74]:
writer.save()